# Demonstrate observables

In [1]:
import mira
from mira.metamodel import Observable, SympyExprStr
import sympy
import torch
from copy import deepcopy as _d
from mira.metamodel.ops import stratify
from sympytorch import SymPyModule
from pyciemss.PetriNetODE.interfaces import (
    load_petri_model,
    setup_petri_model,
    sample_petri,
    calibrate,
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model, 
)
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble
)
import pandas as pd
from pyciemss.utils.interface_utils import convert_to_output_format, csv_to_list, solutions_to_observations
from typing import Dict, List 

Solution = Dict[str, torch.tensor]

TypeError: Too few parameters for typing.Dict; actual 1, expected 2

In [99]:
def get_sampled_value(name, model):
    trace_handler = pyro.poutine.trace(model)
    trace = trace_handler.get_trace()
    if name in trace.nodes and trace.nodes[name]['type'] == 'sample':
        return trace.nodes[name]['value']
    raise KeyError(f'{name} not in trace {trace.nodes}')



In [2]:
from sympy import symbols, exp
from sympy.printing.mathml import mathml

I, S, R, beta, gamma = symbols('I S R beta gamma')

expr = I*S*beta/(S+I+R)
mathml_str = mathml(expr)

print(mathml_str)


<apply><divide/><apply><times/><ci>I</ci><ci>S</ci><ci>&#946;</ci></apply><apply><plus/><ci>I</ci><ci>R</ci><ci>S</ci></apply></apply>


In [75]:
[['Timestep'] + [c for c in sir_observations[0].columns[:-1]]]

[['Timestep', 'Timestep', 'I', 'R']]

## Observe half the true population

In [23]:
from mira.examples.sir import sir_parameterized
tm = _d(sir_parameterized)
tm.initials
symbols = set(tm.initials)
expr = sympy.Add(*[sympy.Symbol(s) for s in symbols])
tm.observables = {'half_population': Observable(
        name='half_population',
        expression=SympyExprStr(expr/2))
    }

tm.observables['half_population'].expression.args[0]

immune_population/2 + infected_population/2 + susceptible_population/2

In [27]:
tm.initials

{'susceptible_population': Initial(concept=Concept(name='susceptible_population', display_name=None, description=None, identifiers={'ido': '0000514'}, context={}, units=None), value=1.0),
 'infected_population': Initial(concept=Concept(name='infected_population', display_name=None, description=None, identifiers={'ido': '0000511'}, context={}, units=None), value=2.0),
 'immune_population': Initial(concept=Concept(name='immune_population', display_name=None, description=None, identifiers={'ido': '0000592'}, context={}, units=None), value=3.0)}

## Compile the observable expression to pytorch

In [30]:
half_population = SymPyModule(expressions=[observable.expression.args[0] 
                                           for observable in tm.observables.values()])

## Expected observable value

In [32]:
expected_total_population = dict(
    infected_population=torch.tensor(1.0),
    immune_population=torch.tensor(0.0),
    susceptible_population=torch.tensor(100.0)
)

expected_half_population = half_population(**expected_total_population)                                
assert expected_half_population == 

In [91]:
import torch
import pyro
num_samples = 2
data_path = 'sir_data.csv'
sir_path = '../../test/models/AMR_examples/sir_typed.json'
timepoints = [0.1, 0.2, 0.3]
raw_sir = load_petri_model(sir_path)

In [ ]:
def observation_model(solution: Solution, var_name: str) -> None:

In [92]:
sir = setup_petri_model(raw_sir, 0.0, dict(S=1000.0, I=1.0, R=0.0))
sir_samples = sample_petri(sir, timepoints , num_samples)
sir_samples
sir_data = csv_to_list(data_path)
sir_data

[(0.1,
  {'I': 13.228653907775879, 'R': 0.05393034219741821, 'S': 987.7174682617188}),
 (0.2,
  {'I': 152.05438232421875, 'R': 0.7183064222335815, 'S': 848.227783203125}),
 (0.3,
  {'I': 701.5635986328125, 'R': 5.340606212615967, 'S': 294.0958557128906})]

In [106]:
get_sampled_value('I_sol', sir)

tensor([])

In [93]:
inferred_parameters = calibrate(sir, sir_data, num_iterations=10)


In [95]:
calibrated_samples = sample_petri(sir, timepoints, num_samples, inferred_parameters)

In [96]:
calibrated_samples

{'beta': tensor([0.0270, 0.0269]),
 'gamma': tensor([0.1341, 0.1364]),
 'I_sol': tensor([[0.9893, 0.9788, 0.9684],
         [0.9891, 0.9783, 0.9677]]),
 'R_sol': tensor([[0.0133, 0.0265, 0.0396],
         [0.0136, 0.0270, 0.0403]]),
 'S_sol': tensor([[999.9982, 999.9947, 999.9920],
         [999.9974, 999.9937, 999.9921]])}

In [ ]:
convert_to_output_format(calibrated_samples, timepoints, 

In [57]:
calibrated_samples2 = load_and_sample_petri_model(sir_path, num_samples, timepoints)
sir_samples2

,timepoint_id,sample_id,beta_param,gamma_param,I_sol,R_sol,S_sol
0,0,0,0.026035,0.113613,13.228654,0.053930,987.717468
1,1,0,0.026035,0.113613,152.054382,0.718306,848.227783
2,2,0,0.026035,0.113613,701.563599,5.340606,294.095856
3,0,1,0.026090,0.122108,13.289371,0.058149,987.652710
4,1,1,0.026090,0.122108,153.239944,0.777267,846.982788
5,2,1,0.026090,0.122108,704.013794,5.773987,291.212341


In [79]:
sir_observations = outputs_to_observations(timepoints, sir_samples2.set_index(['timepoint_id', 'sample_id']))
sir_observations[0].to_csv(data_path, index=False)

In [80]:
sir_observations_0 = pd.read_csv(data_path)
sir_observations_0

,Timestep,I,R,S
0,0.1,13.228654,0.053930,987.717468
1,0.2,152.054382,0.718306,848.227783
2,0.3,701.563599,5.340606,294.095856


In [82]:
calibrated_sir_0 = load_and_calibrate_and_sample_petri_model(
    sir_path, data_path, num_samples, timepoints, num_iterations=10)

In [83]:
calibrated_sir_0

,timepoint_id,sample_id,beta_param,gamma_param,I_sol,R_sol,S_sol
0,0,0,0.027059,0.131313,14.609334,0.066826,986.323792
1,1,0,0.027059,0.131313,179.796600,0.961613,820.242126
2,2,0,0.027059,0.131313,758.326843,7.014094,235.659134
3,0,1,0.026856,0.128114,14.323705,0.064298,986.612000
4,1,1,0.026856,0.128114,173.978394,0.911287,826.110229
5,2,1,0.026856,0.128114,747.708374,6.678317,246.613327


In [20]:
stratified_tm = stratify(tm,
                  key='age',
                  strata=['y', 'o'],
                  structure=[],
                  cartesian_control=True)
stratified_tm

TemplateModel(templates=[ControlledConversion(rate_law=beta_0_0*infected_population_o*susceptible_population_o, type='ControlledConversion', controller=Concept(name='infected_population_o', display_name=None, description=None, identifiers={'ido': '0000511'}, context={'age': 'o'}, units=None), subject=Concept(name='susceptible_population_o', display_name=None, description=None, identifiers={'ido': '0000514'}, context={'age': 'o'}, units=None), outcome=Concept(name='infected_population_o', display_name=None, description=None, identifiers={'ido': '0000511'}, context={'age': 'o'}, units=None), provenance=[]), ControlledConversion(rate_law=beta_0_1*infected_population_y*susceptible_population_o, type='ControlledConversion', controller=Concept(name='infected_population_y', display_name=None, description=None, identifiers={'ido': '0000511'}, context={'age': 'y'}, units=None), subject=Concept(name='susceptible_population_o', display_name=None, description=None, identifiers={'ido': '0000514'}, 